훈련할때 디코더의 입력으로 이전 time-step의 출력 $\hat{y_t}$이 들어가는 것일까요? 

seq2seq의 기본적인 훈련 방식은 추론할때의 방식과 상이합니다.

### 자기회귀 속성

seq2seq의 훈련 방식과 추론 방식의 차이는 근본적으로 자기회귀^{autoregression} (AR)라는 속성때문에 생깁니다. 자귀회귀란 과거의 자신의 값을 참조하여 현재의 값을 추론하는 특징을 가르킵니다.

예를 들어 다음은 전체적인 신경망 기계번역의 수식입니다.

<br></br>
$$
\hat{Y} = argmax_{Y \in Y}P(Y|X) = argmax_{Y in Y} \prod_{i=1}^n P(y_i|X,\hat{y_{<i}}) \\
\text{or} \\
\hat{y_t} = argmax_{y \in Y} P(y_t|X,\hat{y_{<t}};\theta) \\
\text{where}\ X = \{x_1, x_2, \dots, x_n\}, Y = \{y_0, y_1, \dots, y_{m+1}\}, y_0 = BOS
$$
<br></br>

이와 같이 현재 time-step의 출력값 $y_t$는 인코더의 입력 문장 X와 이전 time-step까지의 $y_{<t}$를 조건부로 받아 결점되므로, 과거 자신의 값 $\hat{y_{t-1}}$을 참조합니다. 이러한 점은 과거에 잘못된 예측을 했을 경우 시간이 지날수록 더 큰 잘못된 예측을 할 가능성을 야기하기도 합니다. 또한, 과고의 결과값에 따라 문장의 구성이 바뀔뿐만 아니라, 예측 문장의 길이마저도 바뀝니다. 학습 과정에서는 이미 정답을 알고 있고, 현재 모델의 예측값과 정답과의 차이를 통해 학습하므로, 우리는 자기회귀 속성을 유지한채 훈련할 수 없습니다.

<br></br>
![](./images/10-5-2-teacherforcing.jpg)
<br></br>

따라서 우리는 teacher forcing이라고 불리는 방법을 사용하여 훈련합니다. 훈련할때 각 time-step별 수식은 다음과 같습니다. 조건부에 $\hat{y_{<t}}$가 들어가는 것이 아닌 $y_{<t}$가 들어가는 것이므로, 훈련 시에는 이전 time-step의 출력 $\hat{y_{<t}}$을 현재 time-step의 입력으로 넣어줄 수 없습니다.

<br></br>
$$
L(\theta) = - \frac{1}{N} \sum_{i=1}^N\sum_{t=1}^{m+1} log P(y = y_{i,t}|X, y_{i,<t};\theta) \\
\theta \gets \theta - \lambda \nabla_\theta L(\theta)
$$
<br></br>

또한, 실제 손실 함수를 계산하여 SGD를 수행할때도, `softmax`를 통해 얻은 확률 분포 $logP( \cdot
 | X_i, y_{i,t};\theta)$ 에서 해당 time-step의 argmax 값으 ㅣ확률을 사용하지 않고, Cross entropy 수식에 따라 정답에 해당하는 $y_i$의 인덱스에 있는 log-likelihood $log P(y = y_{i,t}|X_i,y_{i,<t};\theta)$를 사용합니다.
 
중요한 점은 훈련시에는 디코더의 입력으로 이전 time-step의 디코더 출력값이 아닌, 정답 Y가 들어간다는 것입니다. 하지만 추론할때는 정답 Y를 모르기때문에, 이전 time-step에서 추론되어 나온 단어의 원핫 벡터를 디코더의 입력으로 사용합니다. 이렇게 입력을 넣어주는 훈련 방법을 teacher forcing이라고 합니다.

결국 자기회귀 속성으로 인해 teacher forcing 방법을 통해 훈련시킬 수 밖에 없고, 이로 인해 훈련 방법과 추론 방법이 달라집니다.

추론할때는 자기회귀 속성때문에 과거의 자신을 참조해야 합니다. 따라서 이전 time-step의 자기 자신의 상태를 알기 위해서 각 time-step별로 순차적으로 진행해야 합니다. 하지만 훈련할때는 입력값이 정해져 있으므로 모든 time-step을 한번에 계산할 수 있습니다. 그러므로 **input feeding**이 존재하지 않는 디코더는 모든 time-step을 합쳐 수식을 정리할 수 있습니다.

<br></br>
$$
H^{tgt} = RNN_{dec}(emb_{tgt}([BOS;Y[:-1]]),h_n^{src})
$$
<br></br>

디코더의 input feeding은 이전 time-step의 softmax 이전 계층의 값을 단어 임베딩 벡터와 함께 받아야 하므로 앞에서와 같이 모든 time-step을 한번에 계산하는 것은 불가능합니다. 따라서 input feeding이 추가된 디코더의 수식은 다음과 같이 정의됩니다. 이는 teacher forcing을 통한 손실을 만회하기 위함이기도 합니다.

<br></br>
$$
h_t^{tgt} = RNN_{dec}([emb_{tgt}(y_{t-1});\tilde{h_{t-1}^{tgt}}],h_{t-1}^{tgt}) \\
\text{where} h_0^{tgt} = h_n^{src}, y_0 = BOS
$$
<br></br>

이런 자기회귀 속성 및 teacher forcing 방법은 NNLM에도 똑같이 적용되는 문제입니다. 하지만 언어 모델의 경우 PPL은 문장의 확률과 직접적으로 연관이 있기 때문에 큰 문제가 되지 않는 반면, 기계번역에서는 더 큰 문제로 다가옵니다.

### 언어모델의 teacher forcing

<br></br>
![](./images/10-5-3-tflm.jpg)
<br></br>